In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
from astropy.wcs import WCS
import numpy as np
from matplotlib.ticker import MultipleLocator
from scipy.stats import linregress
from matplotlib.colors import LogNorm
import astropy.io.fits as fits
from reproject import reproject_from_healpix
from matplotlib import cm
from matplotlib.colors import Normalize 
from matplotlib import pyplot as plt, patches
from scipy.interpolate import interpn
from reproject import reproject_to_healpix
from astropy_healpix import HEALPix
from astropy.coordinates import SkyCoord, ICRS, Galactic
from astropy import units as units
import matplotlib.cm as cm
import sys
from importlib import reload
import astropy.units as u
import copy
from matplotlib.ticker import MultipleLocator, FuncFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable
import gc
import healpy as hp

### Read in files:

In [ ]:
#hduQ = fits.open('/srv/data/dva/dva_map_test_results/Becky/New_mini_survey_q_cube.fits')
hduQ = fits.open('/srv/data/dva/dva_map_test_results/Anna/Jun02/lowel_lowfreq/cylMap_east_23.q.fits')
hdrQ = hduQ[0].header
dataQ = hduQ[0].data
print(dataQ.shape)

#hduU = fits.open('/srv/data/dva/dva_map_test_results/Becky/New_mini_survey_u_cube.fits')
hduU = fits.open('/srv/data/dva/dva_map_test_results/Anna/Jun02/lowel_lowfreq/cylMap_east_23.u.fits')
hdrU = hduU[0].header
dataU = hduU[0].data
print(dataU.shape)

#hduI = fits.open('/srv/data/dva/dva_map_test_results/Becky/New_mini_survey_i_cube.fits')
hduI = fits.open('/srv/data/dva/dva_map_test_results/Anna/Jun02/lowel_lowfreq/cylMap_east_23.i.fits')
hdrI = hduI[0].header
dataI = hduI[0].data
print(dataI.shape)

freq = []
with open('/srv/data/dva/dva_map_test_results/Becky/ListOfFrequencies_Hz.txt') as f:
    for line in f.readlines():
        freq.append(float(line.split()[0])/1e6)
freq = np.array(freq)
print(freq)
print('')

if ((np.ndim(dataI) != np.ndim(dataQ)) or (np.ndim(dataI) != np.ndim(dataU)) or (np.ndim(dataU) != np.ndim(dataU))):
    print('=========================')
    print('Warning: data array dimensions do not match!!!!')
    print('=========================')
    print('')

if np.ndim(dataI) == 2:
    cube = False
    print('Using 2D data sets')
else:
    cube = True
    print('Using 3D data sets')

### Pick frequency channel:

In [ ]:
####################
freq_choose = 610
el = 'low'
####################

if cube:
    idx = np.where(freq == freq_choose)[0][0]
    print(freq[idx])
else:
    print('Only one frequency in data sets')

### Convert to Kelvin:

In [ ]:
##################
Jy_per_K = 20
##################
dataQ_K = dataQ/Jy_per_K
dataU_K = dataU/Jy_per_K
dataI_K = dataI/Jy_per_K

print(dataI_K.shape)

### Clean up empty parts of map (put in NaNs)

In [ ]:
if cube:
    dataQ_K[:,155:170,:] = np.nan
    dataU_K[:,155:170,:] = np.nan
    dataI_K[:,155:170,:] = np.nan
else:
    dataQ_K[155:170,:] = np.nan
    dataU_K[155:170,:] = np.nan
    dataI_K[155:170,:] = np.nan

dataQ_K[np.where(dataQ_K == 0)] = np.nan
dataU_K[np.where(dataU_K == 0)] = np.nan
dataI_K[np.where(dataI_K == 0)] = np.nan

### Modify header Make 2D header:

In [ ]:
hdrI['BUNIT'] = 'K'
hdrI['OBJECT'] = 'DRAGONS'

hdr_2D = hdrI.copy()
if cube:
    hdr_2D['NAXIS'] = 2
    del hdr_2D['NAXIS3']
    del hdr_2D['CTYPE3']
    del hdr_2D['CRVAL3']
    del hdr_2D['CDELT3']
print(repr(hdr_2D))

In [ ]:
##############
nside = 128 # Note: smoothing a bit - original was nside=64 for now
#############

if cube:

    # Q, U and I just in selected frequency slice:   
    dataQ_K_hpx, footprint = reproject_to_healpix((dataQ_K[idx], hdr_2D),'icrs', nside=nside)
    dataU_K_hpx, footprint = reproject_to_healpix((dataU_K[idx], hdr_2D),'icrs', nside=nside)
    dataI_K_hpx, footprint = reproject_to_healpix((dataI_K[idx], hdr_2D),'icrs', nside=nside)

    # The full array for Stokes I:
    dataI_K_hpx_full = np.empty([len(freq),hp.nside2npix(128)])
    for i in range(0,dataI_K_hpx_full.shape[0]):
        dataI_K_hpx_full[i,:], footprint = reproject_to_healpix((dataI_K[i], hdr_2D),'icrs', nside=nside)

else:

    dataQ_K_hpx, footprint = reproject_to_healpix((dataQ_K, hdr_2D),'icrs', nside=nside)
    dataU_K_hpx, footprint = reproject_to_healpix((dataU_K, hdr_2D),'icrs', nside=nside)
    dataI_K_hpx, footprint = reproject_to_healpix((dataI_K, hdr_2D),'icrs', nside=nside)

### Rotate Stokes I to Galactic

In [ ]:
if cube:

    import healpy
    r = healpy.rotator.Rotator(coord=['C','G'])
    dataI_K_gal_hpx = r.rotate_map_pixel(dataI_K_hpx_full)

else:

    print('Keeping all data in equatorial')

### Make Healpix header (for 2D array)

In [ ]:
hdu_new_hpx = fits.PrimaryHDU()
hdr_hpx = hdu_new_hpx.header

hdr_hpx['PIXTYPE'] = ('HEALPIX ', 'HEALPIX pixelisation')
hdr_hpx['ORDERING'] = ('RING', 'Pixel ordering scheme, either RING or NESTED')
hdr_hpx['NSIDE'] = (nside,     'Resolution parameter of HEALPIX')
hdr_hpx['INDXSCHM'] = ('IMPLICIT', 'Indexing: IMPLICIT or EXPLICIT')
hdr_hpx['OBJECT'] = ('FULLSKY ', 'Sky coverage, either FULLSKY or PARTIAL')
hdr_hpx['COORDSYS'] = ('C', 'Ecliptic, Galactic or Celestial (equatorial)')
#hdr_hpx['NAXIS'] = 1
print(repr(hdr_hpx))

### Write out separate channel Q, U and I Healpix files in Equatorial

In [ ]:
fits.writeto('/srv/data/dva/dva_map_test_results/George/'+el+str(freq_choose)+'_Jul04_q_K.hpx.fits',
             dataQ_K_hpx,header=hdr_hpx,overwrite=True,output_verify='fix')

fits.writeto('/srv/data/dva/dva_map_test_results/George/'+el+str(freq_choose)+'_Jul04_u_K.hpx.fits',
             dataU_K_hpx,header=hdr_hpx,overwrite=True,output_verify='fix')

fits.writeto('/srv/data/dva/dva_map_test_results/George/'+el+str(freq_choose)+'_Jul04_i_K.hpx.fits',
             dataI_K_hpx,header=hdr_hpx,overwrite=True,output_verify='fix')

In [ ]:
dataQ_K_hpx.shape

### Write out separate channel Q, U and I FITS files in Equatorial

In [ ]:
fits.writeto('/srv/data/dva/dva_map_test_results/George/'+el+str(freq_choose)+'_Jul04_q_K.fits',
             dataQ_K,header=hdr_2D,overwrite=True,output_verify='fix')

fits.writeto('/srv/data/dva/dva_map_test_results/George/'+el+str(freq_choose)+'_Jul04_u_K.fits',
             dataU_K,header=hdr_2D,overwrite=True,output_verify='fix')

fits.writeto('/srv/data/dva/dva_map_test_results/George/'+el+str(freq_choose)+'_Jul04_i_K.fits',
             dataI_K,header=hdr_2D,overwrite=True,output_verify='fix')

### Make FITS-CAR Galactic projection for Stokes I cube

In [ ]:
###########
delta = 0.5
##########

# Healpix parameters:
nside = hdr_hpx['NSIDE']
order = hdr_hpx['ORDERING']
hpx = HEALPix(nside=nside, order=order, frame=Galactic())

# Number of pixels along each axis:
numlon = int(360./delta)
numlat = int(180./delta)

# 1D and 2D RA and dec grids:
lon = np.linspace(360, 0, numlon) * u.deg
lat = np.linspace(-90, 90, numlat) * u.deg
l_grid, b_grid = np.meshgrid(lon, lat)

# Set up Astropy coordinate objects
coords = SkyCoord(l_grid, b_grid, frame='galactic')

dataI_K_full = np.empty([len(freq),numlat,numlon])
for i in range(0,len(freq)):
    dataI_K_full[i,:,:] = hpx.interpolate_bilinear_skycoord(coords, dataI_K_gal_hpx[i])
#dataI_K_full = dataI_K_full.reshape((numlat, numlon))[:,::-1]
print(dataI_K_full.shape)

In [ ]:
plt.imshow(dataI_K_full[0,:,:],origin='lower')

In [ ]:
freq_reg = np.linspace(freq[0],freq[-1],35)
plt.scatter(freq,freq_reg)
plt.xlabel('Becky channels')
plt.ylabel('Approximate frequencies')
plt.grid()

hdr_car_3D = fits.Header.fromstring("""
NAXIS   =                    3
CUNIT1  = 'deg     '
CUNIT2  = 'deg     '
CUNIT3  = 'MHz     '
""", sep='\n')

hdr_car_3D['COORDSYS']  = 'galactic'
hdr_car_3D['NAXIS1']  = dataI_K_full.shape[2] 
hdr_car_3D['NAXIS2']  = dataI_K_full.shape[1]
hdr_car_3D['NAXIS3']  = dataI_K_full.shape[0]

hdr_car_3D['CTYPE1']  = 'GLON--CAR'
hdr_car_3D['CRPIX1']  = dataI_K_full.shape[2]/2.+0.5 
hdr_car_3D['CRVAL1']  = 360./2.          
hdr_car_3D['CDELT1']  = -delta

hdr_car_3D['CTYPE2']  = 'GLAT-CAR'
hdr_car_3D['CRPIX2']  = dataI_K_full.shape[1]/2.+0.5
hdr_car_3D['CRVAL2']  = 0.
hdr_car_3D['CDELT2']  = delta

hdr_car_3D['CTYPE3']  = 'FREQ'
hdr_car_3D['CRPIX3']  = 1
hdr_car_3D['CRVAL3']  = freq_reg[0]
hdr_car_3D['CDELT3']  = freq_reg[1] - freq_reg[0]

print(repr(hdr_car_3D))

### Write out Galactic Stokes I FITS cube

In [ ]:
fits.writeto('/srv/data/dragons/DRAGONS_StokesI_June30_K.fits',
             dataI_K_full,header=hdr_car_3D,overwrite=True,output_verify='fix')